In [115]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.inspection import permutation_importance
from sklearn.ensemble import VotingRegressor
import math

In [116]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_test['SalePrice'] = -1
all_data = pd.concat([data_train,data_test]).reset_index()
all_data['Type'] = all_data.apply(lambda x: 'train' if x['SalePrice'] > -1 else 'test',axis=1)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          198 non-null    object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [117]:
# Cleaning
# Primero rellenamos con 'NA' aquellos valores vacíos que no lo son en realidad
NA_features = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
all_data[NA_features] = all_data[NA_features].fillna('NA')
# all_data.loc[(all_data.GarageType == 'NA') & (all_data.GarageCars != 0) ,'GarageCars']




In [118]:
# Data_description.txt nos dice que Functional es typical a menos que Deductions estén garantizadas/justificadas
all_data[['Functional']] = all_data[['Functional']].fillna('Typ')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [119]:
# GarageCars / GarageArea
garage_features = ['Id','GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond','Type']
# El registro con Id 2577 parece no tener Garaje, a pesar de que en su GarageType pone Detchd
# Cambiamos ese valor por NA
all_data.loc[all_data.Id == 2577,'GarageType'] = 'NA'
all_data.loc[(all_data.GarageType != 'NA') & (all_data.GarageYrBlt.isnull()) ,garage_features]



,Id,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,Type
2126,2127,Detchd,NaN,NA,1.0,360.0,NA,NA,test


El registro Id 2127 nos da información contradictoria. Por un lado nos dice que tiene un Garaje tipo Detchd en el que cabe 1 coche y por otro nos dice que no tiene según Finish Qual y Cond.
Tenemos que tomar la decisión de cuál es la info correcta. Optamos por dar por válido el hecho de que tiene garaje. 
Para rellenar Finish vemos que la mayoría de los garajes tipo Detchd son 'Unf'
Más concretamente, vemos que para las casas con GarageCars == 1, la combinación más habitual de Finish, Qual y Cond es 'Unf', 'TA', 'TA' (481)
Además debemos rellenar el GarageYrBlt. Usamos la fecha de última remodelación (1983)

In [120]:
all_data.loc[(all_data.GarageCars == 1)].groupby(['GarageFinish','GarageQual','GarageCond']).count()
all_data.loc[(all_data.GarageCars == 1) & (all_data.GarageFinish == 'Unf') & (all_data.GarageQual == 'TA') & (all_data.GarageCond == 'TA')]
all_data.loc[all_data.Id == 2127,['GarageFinish','GarageQual','GarageCond']] = ['Unf','TA','TA']
all_data.loc[all_data.Id == 2127,['GarageYrBlt']] = all_data.loc[(all_data.Id == 2127),'YearRemodAdd'].values
all_data.loc[(all_data.GarageType != 'NA') & (all_data.GarageYrBlt.isnull()) ,garage_features]
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [121]:
# Rellenamos los nulos de Cars y Area con cero puesto que es un registro que no tiene garaje
all_data[['GarageCars','GarageArea']] = all_data[['GarageCars','GarageArea']].fillna(0)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [122]:

# Nos aseguramos de que sólo los que no tienen garaje no tienen informada la fecha de construcción del garaje
all_data.loc[(all_data.GarageYrBlt.isnull()),garage_features] # 158
all_data.loc[all_data.GarageType == 'NA'] #158
all_data.loc[all_data.GarageType == 'NA'].groupby(['GarageType','GarageQual']).count() # 158
all_data.loc[all_data.GarageType == 'NA'].groupby(['GarageType','GarageQual','GarageCond']).count() # 158
all_data.groupby(['GarageType','GarageQual','GarageCond']).count() # 158

index    Id  MSSubClass  MSZoning  \
GarageType GarageQual GarageCond                                      
2Types     Fa         Fa              1     1           1         1   
                      TA              1     1           1         1   
           TA         TA             21    21          21        21   
Attchd     Fa         Fa              1     1           1         1   
                      Po              1     1           1         0   
                      TA             14    14          14        13   
           Gd         Gd              6     6           6         6   
                      TA             11    11          11        11   
           TA         Fa              4     4           4         4   
                      Gd              6     6           6         6   
                      Po              1     1           1         1   
                      TA           1679  1679        1679      1679   
Basment    Fa         TA              4     4           4         4   
           Gd         TA              1     1           1         1   
           TA         TA             31    31          31        31   
BuiltIn    Fa         TA              3     3           3         3   
           Gd         TA              1     1           1         1   
           TA         TA            182   182         182       182   
CarPort    Fa         Fa              1     1           1         1   
                      TA              1     1           1         1   
           TA         TA             13    13          13        13   
Detchd     Ex         Ex              2     2           2         2   
                      TA              1     1           1         1   
           Fa         Fa             36    36          36        36   
                      Po              7     7           7         7   
                      TA             54    54          54        54   
           Gd         Ex              1     1           1         1   
                      TA              4     4           4         4   
           Po         Fa              1     1           1         1   
                      Po              4     4           4         4   
           TA         Fa             30    30          30        29   
                      Gd              3     3           3         3   
                      Po              1     1           1         1   
                      TA            634   634         634       633   
NA         NA         NA            158   158         158       158   

                                  LotFrontage  LotArea  Street  Alley  \
GarageType GarageQual GarageCond                                        
2Types     Fa         Fa                    1        1       1      1   
                      TA                    0        1       1      1   
           TA         TA                   17       21      21     21   
Attchd     Fa         Fa                    1        1       1      1   
                      Po                    1        1       1      1   
                      TA                   13       14      14     14   
           Gd         Gd                    4        6       6      6   
                      TA                   10       11      11     11   
           TA         Fa                    4        4       4      4   
                      Gd                    2        6       6      6   
                      Po                    1        1       1      1   
                      TA                 1357     1679    1679   1679   
Basment    Fa         TA                    3        4       4      4   
           Gd         TA                    0        1       1      1   
           TA         TA                   23       31      31     31   
BuiltIn    Fa         TA                    2        3       3      3   
           Gd         TA                    1        1       1      1   
           TA      

In [123]:
# Rellenamos la fecha de construcción del garaje con 0
all_data[['GarageYrBlt']] = all_data[['GarageYrBlt']].fillna(0)

In [124]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [125]:
# Basement Cleaning
bsmt_features = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']
all_data.loc[(all_data.BsmtFinSF1.isnull()) | (all_data.BsmtFinSF2.isnull())| (all_data.BsmtUnfSF.isnull())| (all_data.TotalBsmtSF.isnull()),bsmt_features]
all_data[bsmt_features] = all_data[bsmt_features].fillna(0)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [126]:
all_data.loc[(all_data.BsmtQual != 'NA') & (all_data.BsmtFinSF1 == 0) & (all_data.BsmtFinSF2 == 0) & (all_data.BsmtUnfSF == 0), bsmt_features ]
all_data.loc[(all_data.BsmtFinType1 != 'NA') & (all_data.BsmtFinType1 != 'Unf') & (all_data.BsmtFinSF1 == 0), bsmt_features ]
all_data.loc[(all_data.BsmtFinType2 != 'NA') & (all_data.BsmtFinType2 != 'Unf') & (all_data.BsmtFinSF2 == 0), bsmt_features ]
# all_data.loc[((all_data.BsmtFinType2 == 'NA') | (all_data.BsmtFinType2 == 'Unf')) & (all_data.BsmtFinSF2 == 0), bsmt_features ]
all_data.loc[(all_data.BsmtFinSF2 == 0), bsmt_features ].groupby('BsmtFinType2').count()

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
BsmtFinType2,,,,,,,,
BLQ,1,1,1,1,1,1,1,1
NA,79,79,79,79,79,79,79,79
Unf,2492,2492,2492,2492,2492,2492,2492,2492


In [127]:
# Parece que existe un error en el Id 1471. BsmtFinType2 indica BLQ pero BsmtFinSF2 nos dice 0.0. Podría suceder que BsmtFinType2 = 'Unf'
# Sustituimos por 'Unf'
all_data.loc[all_data.Id == 1471, 'BsmtFinType2'] = 'Unf'
all_data.loc[(all_data.BsmtFinSF2 == 0), bsmt_features ].groupby('BsmtFinType2').count()

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
BsmtFinType2,,,,,,,,
NA,79,79,79,79,79,79,79,79
Unf,2493,2493,2493,2493,2493,2493,2493,2493


In [128]:
all_data.loc[all_data.BsmtFinSF1 + all_data.BsmtFinSF2 + all_data.BsmtUnfSF != all_data.TotalBsmtSF,bsmt_features]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF


In [129]:
# Basement Baths
bath_features = bsmt_features + ['BsmtFullBath','BsmtHalfBath']
all_data[['BsmtFullBath','BsmtHalfBath']] = all_data[['BsmtFullBath','BsmtHalfBath']].fillna(0)
all_data.loc[(all_data.BsmtFullBath.isnull()) | (all_data.BsmtHalfBath.isnull()) , bath_features]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath


In [130]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [131]:
# SaleType Cleaning
sale_features = ['MoSold','YrSold','SaleType','SaleCondition']
all_data.loc[all_data.SaleType.isnull(), sale_features]
all_data.loc[all_data.SaleCondition == 'Normal',sale_features].groupby(['SaleType','SaleCondition']).count()
all_data[['SaleType']] = all_data[['SaleType']].fillna('WD')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [132]:
# Kitche Quality Id 1556
# Rellenamos nulos con TA que parece ser el más habitual en las casas con el mismo OverallQual
kit_features = ['KitchenAbvGr','KitchenQual','OverallQual','Id']
all_data.loc[(all_data.OverallQual == 5) & (all_data.YearRemodAdd <= 1953), kit_features].groupby(['KitchenAbvGr','KitchenQual','OverallQual']).count()
# all_data.loc[(all_data.OverallQual == 10) , kit_features].groupby(['KitchenAbvGr','KitchenQual','OverallQual']).count()
all_data[['KitchenQual']] = all_data[['KitchenQual']].fillna('TA')
all_data.loc[all_data.KitchenQual.isnull(), kit_features]

,KitchenAbvGr,KitchenQual,OverallQual,Id


In [133]:
all_data.loc[all_data.Electrical.isnull(),['YearBuilt']]
all_data.loc[all_data.YearBuilt >= 2006].groupby('Electrical').count()
all_data.groupby('Electrical').count()
all_data[['Electrical']] = all_data[['Electrical']].fillna('SBrkr')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 83 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          2919 non-null   object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [134]:
ext_features = ['Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','UniqueExterior']
all_data[['Exterior2nd']] = all_data.apply(lambda x: 'WdShing' if x['Exterior2nd'] == 'Wd Shng' else x['Exterior2nd'],axis=1)
all_data[['Exterior2nd']] = all_data.apply(lambda x: 'CemntBd' if x['Exterior2nd'] == 'CmentBd' else x['Exterior2nd'],axis=1)
all_data[['Exterior2nd']] = all_data.apply(lambda x: 'BrkComm' if x['Exterior2nd'] == 'Brk Cmn' else x['Exterior2nd'],axis=1)
all_data['UniqueExterior'] = all_data.apply(lambda x: 1 if x['Exterior1st'] == x['Exterior2nd'] else 0,axis=1)

all_data.loc[(all_data.Exterior1st.isnull()) | (all_data.Exterior2nd.isnull()),['Exterior1st','Exterior2nd','OverallQual']]
all_data.loc[all_data.OverallQual == 5, ext_features].groupby(['Exterior1st','Exterior2nd']).count()
# La combinación MetalSd,MetalSd es la más habitual
all_data[['Exterior1st','Exterior2nd']] = all_data[['Exterior1st','Exterior2nd']].fillna('MetalSd')

In [135]:


all_data.loc[all_data.MasVnrType.isnull() == False,ext_features].groupby(['UniqueExterior','MasVnrType']).count() #.sort_values('MasVnrArea',ascending=False).head(50)
all_data.loc[(all_data.UniqueExterior == 1) & (all_data.MasVnrType.isnull()),['MasVnrType']] = all_data.loc[(all_data.UniqueExterior == 1) & (all_data.MasVnrType.isnull()),['MasVnrType']].fillna('None')
all_data.loc[all_data.MasVnrType.isnull(),ext_features]
# set(all_data['Exterior2nd'].unique()).symmetric_difference(set(all_data['Exterior1st'].unique()))

,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,UniqueExterior
529,Wd Sdng,Stone,NaN,NaN,Gd,TA,0


In [136]:
all_data.loc[(all_data.Exterior1st == 'Wd Sdng') & (all_data.Exterior2nd == 'Stone'),['MasVnrType']] = all_data.loc[(all_data.Exterior1st == 'Wd Sdng') & (all_data.Exterior2nd == 'Stone'),['MasVnrType']].fillna('None')


In [137]:
all_data[['MasVnrArea']] =all_data[['MasVnrArea']].fillna(0)

In [138]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 84 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           2919 non-null   int64  
 1   Id              2919 non-null   int64  
 2   MSSubClass      2919 non-null   int64  
 3   MSZoning        2915 non-null   object 
 4   LotFrontage     2433 non-null   float64
 5   LotArea         2919 non-null   int64  
 6   Street          2919 non-null   object 
 7   Alley           2919 non-null   object 
 8   LotShape        2919 non-null   object 
 9   LandContour     2919 non-null   object 
 10  Utilities       2917 non-null   object 
 11  LotConfig       2919 non-null   object 
 12  LandSlope       2919 non-null   object 
 13  Neighborhood    2919 non-null   object 
 14  Condition1      2919 non-null   object 
 15  Condition2      2919 non-null   object 
 16  BldgType        2919 non-null   object 
 17  HouseStyle      2919 non-null   o

In [139]:
# Utilities Cleaning
all_data.loc[all_data.Utilities.isnull(), ['Utilities','Electrical','YearBuilt','YearRemodAdd']]
all_data[['Utilities']] = all_data[['Utilities']].fillna('AllPub')

In [140]:
all_data.loc[all_data.Street == 'Grvl'].groupby('MSZoning',).count()
all_data.loc[all_data.Street == 'Pave'].groupby('MSZoning',).count()
# all_data.loc[all_data.MSZoning.isnull()]
# all_data['MSZoning'].unique()
all_data.loc[all_data.Street == 'Grvl', ['MSZoning']] = all_data.loc[all_data.Street == 'Grvl', ['MSZoning']].fillna('C (all)')
all_data.loc[all_data.Street != 'Grvl', ['MSZoning']] = all_data.loc[all_data.Street != 'Grvl', ['MSZoning']].fillna('RL')

In [141]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 84 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           2919 non-null   int64  
 1   Id              2919 non-null   int64  
 2   MSSubClass      2919 non-null   int64  
 3   MSZoning        2919 non-null   object 
 4   LotFrontage     2433 non-null   float64
 5   LotArea         2919 non-null   int64  
 6   Street          2919 non-null   object 
 7   Alley           2919 non-null   object 
 8   LotShape        2919 non-null   object 
 9   LandContour     2919 non-null   object 
 10  Utilities       2919 non-null   object 
 11  LotConfig       2919 non-null   object 
 12  LandSlope       2919 non-null   object 
 13  Neighborhood    2919 non-null   object 
 14  Condition1      2919 non-null   object 
 15  Condition2      2919 non-null   object 
 16  BldgType        2919 non-null   object 
 17  HouseStyle      2919 non-null   o

In [142]:
from custom_functions import get_resume, information_gain
all_data[['LotFrontage']].var()
gains = []
for x in all_data.columns:
    if x != 'LotFrontage' and pd.api.types.is_object_dtype(all_data[x].dtype):
        gains.append((x,information_gain(all_data,'LotFrontage',x,norm=True)))
# print(gains)
gains.sort(key=lambda x: x[1], reverse = True)
gains
# print("Information gain: {}".format(information_gain(all_data,'LotFrontage','BldgType', norm=True)))
# get_resume(all_data,"LotFrontage",'BldgTypeNeighborhood')

lf_means = all_data[['LotFrontage','BldgType']].groupby('BldgType').mean().to_dict()
all_data['LotFrontage'] = all_data.apply(lambda x: lf_means['LotFrontage'][x['BldgType']] if math.isnan(x['LotFrontage']) else x['LotFrontage'], axis=1)
#  all_data[['LotFrontage']] = all_data[['LotFrontage']].fillna(all_data['LotFrontage'].mean())


In [143]:
# New Features
all_data['LotAreaNorm'] = np.log(all_data.LotArea + 1)
all_data['FireplacesNorm'] = np.log(all_data.Fireplaces + 1)
all_data['OpenPorchSFNorm'] = np.log(all_data.OpenPorchSF + 1)
all_data["BsmtFinType1BsmtFinType2"] = all_data["BsmtFinType1"]+all_data["BsmtFinType2"]
all_data["HeatingHeatingQC"] = all_data["Heating"] + all_data["HeatingQC"]
all_data["1stFlrSF2ndFlrSF"] = all_data["1stFlrSF"] + all_data["2ndFlrSF"]
all_data["TotalBaths"] = all_data["HalfBath"] + all_data["FullBath"] + all_data["BsmtHalfBath"] + all_data["BsmtFullBath"]
all_data['OverallQualOverallCond'] = all_data['OverallQual'] * all_data['OverallCond']
tags = ['Po','Fa','TA','Gd','Ex']
all_data['ExterQualExterCond'] = all_data.apply(lambda x: (tags.index(x['ExterQual'])+1)*(tags.index(x['ExterCond'])+1),axis=1)
all_data['LotDepth'] = all_data['LotArea'] / all_data['LotFrontage']
# columns_dict = {}
# treatments = []
# for x in all_data.columns:
#     if pd.api.types.is_numeric_dtype(all_data[x].dtype):
#         treatments.append('num')
#     else:
#         treatments.append('cat')
# columns_dict = {'feature':all_data.columns, 'treatment':treatments}
# columns_df = pd.DataFrame(columns_dict)
# columns_df['good'] = 0
# columns_df['bad'] = 0
# columns_df.to_csv('columns_df.csv')

In [144]:
columns_df = pd.read_csv('columns_df.csv', index_col = 'index')
drop_features = columns_df.loc[columns_df.treatment == 'drop','feature'].unique()
cat_features = columns_df.loc[columns_df.treatment == 'cat','feature'].unique()
num_features = columns_df.loc[columns_df.treatment == 'num','feature'].unique()
pass_features = columns_df.loc[columns_df.treatment == 'pass','feature'].unique()
cat_features

array(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition', 'BsmtFinType1BsmtFinType2',
       'HeatingHeatingQC'], dtype=object)

In [145]:
all_data['Type'] = all_data.apply(lambda x: 0 if x['Type'] == 'train' or x['Type'] == 0 else 1,axis=1)
num_pipeline = Pipeline([
    ("1",RobustScaler()),
    ("2",MinMaxScaler()),
])
pipeline = ColumnTransformer([    
    ('pass','passthrough',pass_features),
    # ('num',RobustScaler(),num_features),
    ('num',num_pipeline,num_features),
    ('cat',OneHotEncoder(sparse=False),cat_features)
    # ('cat',OrdinalEncoder(),cat_features)
])
all_data_prep_array = pipeline.fit_transform(all_data)
ohe_features = pipeline.named_transformers_['cat'].get_feature_names()
all_features = np.concatenate([pass_features,num_features,ohe_features])
all_data_prep = pd.DataFrame(all_data_prep_array,columns = all_features)
# all_features = np.concatenate([pass_features,num_features,cat_features])
all_data_prep = pd.DataFrame(all_data_prep_array,columns = all_features)
all_data_prep.describe()

,SalePrice,Type,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,x45_GasATA,x45_GasWEx,x45_GasWFa,x45_GasWGd,x45_GasWTA,x45_GravFa,x45_OthWFa,x45_WallFa,x45_WallPo,x45_WallTA
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,90491.088386,0.499829,0.041450,0.719658,0.571075,0.063373,0.078184,0.032481,0.239974,0.172081,...,0.287085,0.000685,0.001713,0.001028,0.005824,0.003083,0.000685,0.001370,0.000343,0.000343
std,106496.804191,0.500086,0.036865,0.219503,0.348239,0.111784,0.080724,0.110864,0.188181,0.072196,...,0.452479,0.026171,0.041359,0.032048,0.076105,0.055451,0.026171,0.036999,0.018509,0.018509
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,0.000000,0.028877,0.590580,0.250000,0.000000,0.000000,0.000000,0.094178,0.129787,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,34900.000000,0.000000,0.038108,0.731884,0.716667,0.000000,0.065202,0.000000,0.199914,0.161866,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,163000.000000,1.000000,0.048003,0.934783,0.900000,0.102187,0.129872,0.000000,0.344606,0.213093,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,755000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [146]:
train = all_data_prep.loc[all_data_prep.Type == 0,:]
test = all_data_prep.loc[all_data_prep.Type == 1,:]
test_prepared = test.drop(['SalePrice','Type'],axis=1)
X = train.drop(['SalePrice','Type'],axis=1)
y = train['SalePrice']

In [147]:
corr_columns = train.corr().sort_values(by='SalePrice',ascending = False).reset_index()['index']
corr_columns.drop(0, inplace= True)
corr_columns

1      1stFlrSF2ndFlrSF
2             GrLivArea
3            GarageCars
4            GarageArea
5           TotalBsmtSF
             ...       
360              x19_TA
361                Type
362              x0_150
363          x44_RecGLQ
364          x45_WallPo
Name: index, Length: 364, dtype: object

In [148]:
# clf = SVR(C=100400 ,degree=2,epsilon=2500, kernel='poly')
clf = XGBRegressor(objective="reg:squarederror")
# clf = LGBMRegressor()
# # cross_val_score(clf,X.values,y,scoring='neg_mean_squared_log_error', cv=10,verbose=1)
# clf.fit(X,y)
# feature_importance = pd.DataFrame({"feature": X.columns, "importance":map(lambda x: round(x,4),clf.feature_importances_)}).sort_values(by='importance',ascending=False)
# feature_importance
# lr_clf = LinearRegression()
# params = [{"max_depth":[3,6,7,10,60],"lambda":[1,1.5,2],"estimators":[20,30,100,300,600],"gamma":[0,0.01,0.03,0.1] }]
# params = [{"C":[10,30,60,100,600,1000100,1000000],"degree":[1,2,3,4,5]}]
# params = [{"max_depth":[3,6,10,60,100,500,-1],"learning_rate":[0.01,0.05,0,1],"estimators":[20,30,100,300,600],"num_leaves":[15,31,63,127] }]
# grid = GridSearchCV(clf,params,scoring="neg_mean_squared_log_error", cv=8, verbose=1,n_jobs=-1)
# grid.fit(X,y)
# clf_best = grid.best_estimator_
# print("Best:", grid.best_params_)
print(math.sqrt(-1*cross_val_score(clf,X,y,cv=6,n_jobs=-1,scoring='neg_mean_squared_log_error').mean()))

0.13158853909900822


In [149]:
clf = XGBRegressor(objective="reg:squarederror")
clf2 = SVR(kernel='poly')
params = [{"max_depth":[3,5,7],"estimators":[3,5,10,20,50], "lambda":[1,1.5,2], "min_child_weight":[0.5,1,1.5]}]
params2 = [{"C":[100090,100420,100030,100040],"epsilon":[6,7,9,10,40,200], "degree":[2,3] }]
grid = GridSearchCV(clf,params,scoring="neg_mean_squared_log_error", cv=6, verbose=1,n_jobs=-1)
grid.fit(X,y)
clf_best1 = grid.best_estimator_
print("Best1: {} | Error: {}".format(grid.best_params_, math.sqrt(-1*grid.best_score_.mean())))
grid2 = GridSearchCV(clf2,params2,scoring="neg_mean_squared_log_error", cv=6, verbose=1,n_jobs=-1)
grid2.fit(X,y)
clf_best2 = grid2.best_estimator_
print("Best1: {} | Error: {}".format(grid2.best_params_, math.sqrt(-1*grid2.best_score_.mean())))

Fitting 6 folds for each of 135 candidates, totalling 810 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed:  1.4min finished
Best1: {'estimators': 3, 'lambda': 1, 'max_depth': 5, 'min_child_weight': 1.5} | Error: 0.12771776019575146
Fitting 6 folds for each of 48 candidates, totalling 288 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:   35.9s finished
Best1: {'C': 100420, 'degree': 2, 'epsilon': 200} | Error: 0.13186737939328172


In [153]:
r = permutation_importance(clf_best1,X, y, n_repeats=20, random_state=42, n_jobs=-1)
feature_importance = pd.DataFrame({"feature": X.columns, "importance":map(lambda x: round(x,4),r.importances_mean)}).sort_values(by='importance',ascending=False)
feature_importance.loc[feature_importance.importance > 0.0015]

,feature,importance
32,1stFlrSF2ndFlrSF,0.0757
34,OverallQualOverallCond,0.0568
17,GarageCars,0.0499
7,TotalBsmtSF,0.0484
11,GrLivArea,0.0261
4,BsmtFinSF1,0.0193
1,YearBuilt,0.0165
33,TotalBaths,0.0148
0,LotArea,0.0138
200,x22_Ex,0.0115


In [154]:
final_clf = VotingRegressor([('clf1',clf_best1),('clf2',clf_best2)])
finals = X.columns.values
X_train_finals = X[finals]
error = -1 * cross_val_score(final_clf,X_train_finals.values,y,scoring='neg_mean_squared_log_error', cv=6).mean()
min_error = error    
removed = []
finals_temp = []
i = 0
for f in feature_importance['feature']:
    removed.append(f)
    X_train_finals = X.drop(removed,axis=1)
    error = -1 * cross_val_score(final_clf,X_train_finals.values,y,scoring='neg_mean_squared_log_error', cv=6).mean()
    if min_error > error:
        min_error = error
    else:        
        removed.remove(f) 
    print("Sqrt Error: {} | Features: {} | Min Error: {} | Step: {}: {}/{}".format(math.sqrt(error),X_train_finals.shape[1],math.sqrt(min_error),f,i+1,X.shape[1]))  
    i += 1   


8025423702227 | Features: 310 | Min Error: 0.1188025423702227 | Step: x44_BLQBLQ: 177/363
Sqrt Error: 0.11886428206299922 | Features: 309 | Min Error: 0.1188025423702227 | Step: x44_BLQGLQ: 178/363
Sqrt Error: 0.11878324337558914 | Features: 309 | Min Error: 0.11878324337558914 | Step: x44_BLQLwQ: 179/363
Sqrt Error: 0.11879023166603483 | Features: 308 | Min Error: 0.11878324337558914 | Step: x44_BLQRec: 180/363
Sqrt Error: 0.11881280088394335 | Features: 308 | Min Error: 0.11878324337558914 | Step: x44_BLQUnf: 181/363
Sqrt Error: 0.11878767835780407 | Features: 308 | Min Error: 0.11878324337558914 | Step: x44_GLQALQ: 182/363
Sqrt Error: 0.11877748326610647 | Features: 308 | Min Error: 0.11877748326610647 | Step: x44_GLQBLQ: 183/363
Sqrt Error: 0.11877012228828616 | Features: 307 | Min Error: 0.11877012228828616 | Step: x44_LwQBLQ: 184/363
Sqrt Error: 0.11877084281322728 | Features: 306 | Min Error: 0.11877012228828616 | Step: x44_LwQALQ: 185/363
Sqrt Error: 0.11875537993052976 | Featu

In [156]:
X_final = X.drop(removed,axis=1)
final_clf.fit(X_final,y)
y_pred = final_clf.predict(test_prepared.drop(removed,axis=1))
data_test['SalePrice'] = y_pred
data_test[['Id','SalePrice']].to_csv('result4.csv',index=False)

In [108]:
removed
# # test_prepared[['x0_20','x0_60','x0_120']].head(20)
# # data_test.head(20)
# finals2 = feature_importance.iloc[:,0].unique()
# print(math.sqrt(-1*cross_val_score(clf,X[finals2],y,cv=8,n_jobs=-1,scoring='neg_mean_squared_log_error').mean()))
# clf.fit(X[finals2],y)
# y_pred = clf.predict(test_prepared[finals2])
# data_test['SalePrice'] = y_pred
# data_test[['Id','SalePrice']].to_csv('result4.csv',index=False)

['TotalBaths',
 'x16_MetalSd',
 'x0_70',
 'x19_Fa',
 'x4_IR1',
 'x32_Maj2',
 'x32_Min1',
 'x26_Rec',
 'x43_Alloca',
 'x44_BLQGLQ',
 'x10_RRAn',
 'x0_75',
 'x12_TwnhsE',
 'x13_1Story',
 'x17_Stucco',
 'x17_WdShing',
 'x21_BrkTil',
 'x22_Fa',
 'x24_Mn',
 'x16_WdShing']

Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   

  Alley Lot Shape Land Contour  ... Pool Area Pool QC Fence Misc Feature  \
0   NaN       IR1          Lvl  ...         0     NaN   NaN          NaN   

  Misc Val Mo Sold Yr Sold Sale Type  Sale Condition  SalePrice  
0        0       5    2010       WD           Normal     215000  

[1 rows x 82 columns]
      Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
171  172          20       RL        141.0    31770   Pave   NaN      IR1   

    LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal  \
171         Lvl    AllPub  ...        0    NaN   NaN         NaN       0   

    MoSold YrSold  SaleType  SaleCondition  SalePrice  
171      5   2010        WD         Normal     215000  

[1 rows x 81 columns]
